In [ ]:
import pandas as pd;
from pandas import DataFrame, Series;
import datetime;
import json;

In [ ]:
import sys
sys.path.insert(0, '../DAL')

import dataService
service = dataService.DataService()
r_r = service.review_dates()

In [ ]:
#Compute date; remove unnecessary columns & reviews
# r_r = DataFrame(dataService.DataService.review_dates()[:100])
lastDay = datetime.datetime(2015, 12, 24)
r_r['reference_date'] = r_r['reference_date'].map(lambda x: (lastDay - datetime.datetime.strptime(x, '%Y-%m-%d')).days)
r_r['review_date'] = r_r['review_date'].map(lambda x: (lastDay - datetime.datetime.strptime(x, '%Y-%m-%d')).days)

In [ ]:
r_r.head()

In [ ]:
r_r['days_since_review'] = r_r.apply(lambda x: x['reference_date'] - x['review_date'], axis=1)
r_r['time_distribution'] = r_r.apply(
    lambda x: 1 if (x['days_since_review'] > 0) 
    else (
        0 if (x['days_since_review'] + x['reference_date'] >= 0)
        else -1
    ), axis=1)
# then -1)
r_r = r_r[['authority_id', 'reference_id', 'user_id', 'review_id', 'authority_stars', 'time_distribution', 'validation_token']]
r_r = DataFrame(r_r.loc[r_r['time_distribution'] >= 0])

In [ ]:
#Add distances
u_u = pd.read_json('../full_dataset/user_distance_matrix.json')
r_r = pd.merge(r_r, u_u, on='validation_token')
u_u = None

In [ ]:
r_r.head()

In [ ]:
distribution = r_r.groupby(['authority_id', 'time_distribution', 'distance', 'authority_stars']).size().reset_index()

In [ ]:
distribution.to_json('../full_dataset/visit_distribution.json')

In [ ]:
distribution = r_r.groupby(['reference_id', 'time_distribution', 'distance', 'authority_stars']).size().reset_index()
distribution.to_json('../full_dataset/visit_distribution.json')